# **3. 레이블 데이터 (full_intent_labels.tsv)를 이용한 AI 모의 면접 시스템 구축**

## 1. 데이터 준비 및 모델 로드

- BERT 토크나이저와 사전 훈련된 모델을 로드
- 직무별 키워드, 카테고리 설명, 감정 사전 등을 정의

## 2. 주요 함수

- `predict()`: BERT 모델을 사용하여 문장의 카테고리, 표현, 설명, 직무를 예측
- `improved_sentiment_analysis()`: 감정 사전을 이용해 문장의 감정 점수를 계산
- `analyze_response()`: 문장에 대해 종합적인 분석을 수행
  - BERT 모델 예측
  - TF-IDF를 이용한 중요 단어 추출
  - 직무별 키워드 매칭
  - 감정 분석

## 3. 피드백 생성

- `generate_feedback()`: 분석 결과를 바탕으로 구조화된 피드백을 생성
  - 기본 정보 (카테고리, 역량/특성, 관련 직무)
  - 강점 (매칭된 키워드, 긍정적 표현)
  - 개선점 (누락된 키워드 제안)
  - 구체적인 피드백 (중요 단어에 기반한 상세 조언)
  - 결론 (전반적인 평가 및 조언)

<직무별 키워드 참고 자료>
- 한국직업능력개발원의 NCS(국가직무능력표준) 자료:
https://www.ncs.go.kr/
- LinkedIn의 "The Skills Companies Need Most" 연간 보고서:
https://business.linkedin.com/talent-solutions/blog/trends-and-research/2020/most-in-demand-hard-and-soft-skills

<감정 사전 참고 자료>
- 국립국어원의 "한국어 감성 사전" 프로젝트:
https://www.korean.go.kr/front/board/boardStandardView.do?board_id=4&mn_id=17&b_seq=464
- KOSAC(Korean Sentiment Analysis Corpus) 프로젝트:
http://word.snu.ac.kr/kosac/

<불용어 목록 참고 자로>
- 국립국어원의 "한국어 말뭉치" 자료:
https://corpus.korean.go.kr/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install koreanize_matplotlib
!fc-list | grep NanumGothic
!apt-get install fonts-nanum*dd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 54.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package fonts-nanum*dd
E: Couldn't find any package by glob 'fonts-nanum*dd'
E: Couldn't find any package by regex 'fonts-nanum*dd'


In [3]:
import koreanize_matplotlib
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [4]:
!pip uninstall tensorflow
!pip install tensorflow==2.15.0
!pip install keras==2.15.0
!pip install transformers==4.35.0
!pip install keras-tuner

Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Would remove:
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.17.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-2.17.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 113.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
  At

In [5]:
import tensorflow as tf
import keras
import transformers

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print(f"Transformers version: {transformers.__version__}")

TensorFlow version: 2.15.0
Keras version: 2.15.0
Transformers version: 4.35.0


In [6]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 32.6 MB/s eta 0:00:00


In [7]:
!pip install scikit-learn

In [8]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from transformers import BertTokenizer, TFBertModel
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
# BERT 토크나이저 및 모델 로드
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")
custom_objects = {'TFBertModel': TFBertModel}
with tf.keras.utils.custom_object_scope(custom_objects):
    model = load_model('/content/drive/MyDrive/Colab Notebooks/Project(InterviewData-Analysis)/Models/best_intent_model.h5')

In [10]:
# 한국어 형태소 분석기 초기화
okt = Okt()

# 설정
max_len = 128  # 최대 문장 길이

In [11]:
# 레이블 데이터 로드
full_intent_labels = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Project(InterviewData-Analysis)/data/full_intent_labels.tsv", sep="\t", encoding="utf8")
full_intent_labels

,category,expression,label,description,job
0,attitude,d_creative,0,창의적 사고,예술/디자인
1,attitude,d_express,1,표현력,예술/디자인
2,attitude,d_sens,2,감수성,예술/디자인
3,attitude,f_cons,3,성실성,제조/생산
4,attitude,f_pat,4,끈기,R&D(개발/제조)
5,attitude,f_resp,5,책임감,제조/생산
6,attitude,i_hwsw_prfi,6,HW/SW 이해,ICT
7,attitude,i_secty,7,정보보안,ICT
8,attitude,i_tech_orien,8,ICT 기술 지향성,ICT
9,attitude,m_critical,9,비판적 사고,영업/판매


In [12]:
# 직무별 키워드 정의
job_keywords = {
    "경영/사무": ["분석", "전략", "의사소통", "성과", "브랜드", "세일즈", "대인관계", "기획", "관리", "목표", "리더십"],
    "공공/서비스": ["고객", "서비스", "윤리", "주도성", "스트레스", "대응", "VOC", "민원", "공익", "정책", "협력"],
    "R&D(개발/제조)": ["연구", "기술", "문제해결", "전문성", "정보수집", "방법론", "실적", "혁신", "개발", "실험", "분석"],
    "예술/디자인": ["창의", "트렌드", "감각", "표현", "브랜드", "프로세스", "선호", "미적", "시각", "컨셉", "아이디어"],
    "영업/판매": ["조율", "회계", "행정", "목표", "총괄", "비판적", "경험", "협상", "고객관리", "마케팅", "실적"],
    "ICT": ["프로그래밍", "개발", "장애대응", "기술", "보안", "HW", "SW", "네트워크", "데이터", "알고리즘", "시스템"],
    "제조/생산": ["교대근무", "생산계획", "장비운용", "성실성", "책임감", "품질관리", "안전", "효율성", "공정", "재고관리"]
}

# 카테고리 설명
category_descriptions = {
    "attitude": "태도 관련 역량",
    "background": "배경 및 경험",
    "etc": "기타 관련 사항",
    "personality": "성격 및 개인적 특성",
    "technology": "지식/기술 관련 역량"
}

# 한국어 감정 사전
korean_sentiment_dict = {
    '좋은': 2, '훌륭한': 2, '열정적인': 2, '성공적인': 2, '효과적인': 2, '향상': 2, '달성': 2,
    '노력': 1, '원활': 1, '조율': 1, '협력': 1, '개선': 1, '발전': 1, '성장': 1,
    '높였습니다': 2, '기여했습니다': 2, '적용': 1, '새로운': 1, '창의적': 2,
    '나쁜': -2, '어려운': -1, '힘든': -1, '실패한': -2, '비효율적인': -2, '부족한': -1
}

# 불용어 목록 정의
stop_words = set([
    '있다', '없다', '하다', '되다', '이다', '돼다', '않다', '같다', '이', '그', '저', '것', '들', '등', '를', '을', '에', '의', '과', '로', '으로',
    '입니다', '습니다', '였습니다', '있습니다', '있었습니다', '저는', '가', '이', '은', '는', '들', '좀', '잘', '걍', '과', '도', '를', '으로', '자',
    '에', '와', '한', '하다', '가지다', '사용하다', '되다', '해보다', '대한', '통해', '후', '가다', '오다', '말하다', '생각하다', '보다', '알다', '그렇다',
    '이렇다', '그러다', '이러다', '그리고', '그래서', '그러나', '하지만', '또는', '또한', '그런데', '그러면', '그럼', '즉', '또', '및', '혹은', '거나',
    '뿐만 아니라', '게다가', '어떤', '무슨', '기반', '위해', '정도', '만큼', '때문', '계속', '가능', '관련', '필요', '등등', '때', '중', '많이', '역시',
    '좋다', '나쁘다', '작다', '크다', '높다', '낮다', '있어서', '같아서', '어떻게', '그런', '이런', '그냥', '바로', '일단', '혹시', '가장', '제일',
    '모두', '전부', '항상', '종종', '자주', '거의', '매우', '상당히', '너무', '약간', '조금', '따라서', '결과적으로', '하며', '더', '받', '맡', '얻'
])

In [13]:
# 주어진 문장에 대해 예측을 수행하는 함수
def predict(sentence):
    inputs = tokenizer(
        sentence,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )

    out = model({'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask']})
    logits = out.numpy()
    predicted_class = np.argmax(logits, axis=1)[0]

    prediction_info = full_intent_labels.iloc[predicted_class]
    category = prediction_info['category']
    expression = prediction_info['expression']
    description = prediction_info['description']
    job = prediction_info['job']

    return category, expression, description, job

In [14]:
# 주어진 텍스트의 감정 점수를 계산하는 함수
def improved_sentiment_analysis(text):
    words = okt.morphs(text)
    score = sum(korean_sentiment_dict.get(word, 0) for word in words)

    positive_patterns = ['을 높였습니다', '에 기여했습니다', '을 달성했습니다', '을 쌓아왔습니다', '을 키울 수 있었습니다']
    for pattern in positive_patterns:
        if pattern in text:
            score += 2

    return score

In [15]:
# 주어진 문장에 대해 종합적인 분석을 수행하는 함수
def analyze_response(sentence):
    category, expression, description, job = predict(sentence)

    # 형태소 분석 및 불용어 제거
    morphs = okt.morphs(sentence)
    meaningful_words = [word for word in morphs if word not in stop_words]

    # 직무별 키워드 매칭 (불용어 제거 후)
    matched_keywords = [keyword for keyword in job_keywords.get(job, []) if keyword in meaningful_words]
    all_matched_keywords = [keyword for job_kw in job_keywords.values() for keyword in job_kw if keyword in meaningful_words]

    # 감정 분석 (불용어 제거 후)
    sentiment = improved_sentiment_analysis(' '.join(meaningful_words))

    return {
        'category': category,
        'expression': expression,
        'job': job,
        'description': description,
        'matched_keywords': matched_keywords,
        'all_matched_keywords': all_matched_keywords,
        'sentiment': sentiment,
        'meaningful_words': meaningful_words  # 분석에 사용된 의미 있는 단어들
    }

In [16]:
# # 분석 결과를 바탕으로 피드백을 생성하는 함수
# def generate_feedback(analysis_result):
#     category = analysis_result['category']
#     expression = analysis_result['expression']
#     job = "전체" if analysis_result['job'] == "전 직무" else analysis_result['job']
#     description = analysis_result['description']

#     # feedback = f"답변 카테고리: {category_descriptions.get(category, category)}\n"
#     # feedback += f"관련 역량/특성: {description} (표현: {expression})\n"
#     # feedback += f"관련 직무: {job}\n\n"

#     feedback= ""
#     # 1. 내용 관련 피드백
#     # feedback += "1. 내용 분석:\n"
#     for keyword in analysis_result['matched_keywords']:
#         feedback += f"- {keyword}와(과) 관련된 내용을 언급하여 {job} 직무에 대한 이해를 보여주었습니다.\n"

#     other_keywords = set(analysis_result['all_matched_keywords']) - set(analysis_result['matched_keywords'])
#     if other_keywords:
#         feedback += f"- 다음 키워드들도 언급되어 다양한 역량을 보여주었습니다: {', '.join(other_keywords)}\n"

#     missing_keywords = set(job_keywords.get(job, [])) - set(analysis_result['matched_keywords'])
#     if missing_keywords:
#         feedback += f"- {job} 직무와 더 관련성을 높이기 위해 다음 키워드들을 고려해보세요: {', '.join(list(missing_keywords)[:3])}\n"

#     # 2. 태도 관련 피드백
#     # feedback += "\n2. 태도 분석:\n"
#     if analysis_result['sentiment'] > 3:
#         feedback += "- 매우 긍정적이고 열정적인 태도가 잘 드러납니다.\n"
#     elif analysis_result['sentiment'] > 0:
#         feedback += "- 긍정적인 태도가 느껴집니다.\n"
#     elif analysis_result['sentiment'] == 0:
#         feedback += "- 중립적인 태도로 답변하셨습니다. 조금 더 적극적인 표현을 사용해보는 것은 어떨까요?\n"
#     else:
#         feedback += "- 답변에 더 긍정적인 표현을 사용해 보는 것이 좋겠습니다.\n"

#     # 3. 구체성 관련 피드백
#     # feedback += "\n3. 구체성 분석:\n"
#     if len(analysis_result['meaningful_words']) > 30:
#         feedback += "- 답변이 상당히 구체적이고 상세합니다. 좋은 인상을 줄 수 있습니다.\n"
#     elif len(analysis_result['meaningful_words']) > 20:
#         feedback += "- 적절한 수준의 구체성을 보여주고 있습니다.\n"
#     else:
#         feedback += "- 답변을 조금 더 구체화하면 좋겠습니다. 특정 경험이나 예시를 추가해보세요.\n"

#     # 4. 언어 사용 관련 피드백
#     # feedback += "\n4. 언어 사용 분석:\n"
#     unique_words = set(analysis_result['meaningful_words'])
#     if len(unique_words) > 30:
#         feedback += "- 다양한 어휘를 사용하여 풍부한 답변을 제시했습니다. 이는 해당 분야에 대한 깊은 이해를 보여줍니다.\n"
#     elif len(unique_words) > 20:
#         feedback += "- 적절한 수준의 어휘를 사용하고 있습니다.\n"
#     else:
#         feedback += "- 더 다양한 전문 용어와 표현을 사용하여 답변을 풍부하게 만들어보세요.\n"

#     # 5. 종합 피드백 및 개선 제안 (sttresult1, sttresult2)
#     feedback += f"- 이 답변은 전반적으로 {job} 직무와 관련된 경험과 기술을 잘 보여주고 있습니다.\n"
#     feedback += f"- 앞으로의 면접에서는 {job} 직무에 중요한 기술이나 역량을 어떻게 개발하고 적용했는지 구체적인 예시와 함께 설명하고, 긍정적이고 열정적인 태도를 유지하면서 전문성을 잘 드러내는 어휘를 사용하세요.\n"

#     return feedback

In [17]:
# # AI 모의 면접을 시작하는 함수
# def interview_simulation():
#     print("AI 모의 면접을 시작합니다. '종료'라고 입력하면 면접이 종료됩니다.")
#     while True:
#         user_input = input("\n답변을 입력하세요: ")
#         if user_input.lower() == '종료':
#             print("면접이 종료되었습니다. 수고하셨습니다.")
#             break

#         analysis = analyze_response(user_input)
#         feedback = generate_feedback(analysis)
#         print("\n피드백:")
#         print(feedback)

# # 실행
# if __name__ == "__main__":
#     interview_simulation()

In [18]:
# 분석 결과를 바탕으로 피드백을 생성하는 함수 (aifeedback으로 던져줄 값)
def generate_feedback(analysis_result):
    category = analysis_result['category']
    expression = analysis_result['expression']
    job = "전체" if analysis_result['job'] == "전 직무" else analysis_result['job']
    description = analysis_result['description']

    feedback = f"답변 카테고리: {category_descriptions.get(category, category)}\n"
    feedback += f"관련 역량/특성: {description} (표현: {expression})\n"
    feedback += f"관련 직무: {job}\n\n"

    # 1. 내용 관련 피드백
    feedback += "1. 내용 분석:\n"
    for keyword in analysis_result['matched_keywords']:
        feedback += f"- {keyword}와(과) 관련된 내용을 언급하여 {job} 직무에 대한 이해를 보여주었습니다.\n"

    other_keywords = set(analysis_result['all_matched_keywords']) - set(analysis_result['matched_keywords'])
    if other_keywords:
        feedback += f"- 다음 키워드들도 언급되어 다양한 역량을 보여주었습니다: {', '.join(other_keywords)}\n"

    missing_keywords = set(job_keywords.get(job, [])) - set(analysis_result['matched_keywords'])
    if missing_keywords:
        feedback += f"- {job} 직무와 더 관련성을 높이기 위해 다음 키워드들을 고려해보세요: {', '.join(list(missing_keywords)[:3])}\n"

    # 2. 태도 관련 피드백
    feedback += "\n2. 태도 분석:\n"
    if analysis_result['sentiment'] > 3:
        feedback += "- 매우 긍정적이고 열정적인 태도가 잘 드러납니다.\n"
    elif analysis_result['sentiment'] > 0:
        feedback += "- 긍정적인 태도가 느껴집니다.\n"
    elif analysis_result['sentiment'] == 0:
        feedback += "- 중립적인 태도로 답변하셨습니다. 조금 더 적극적인 표현을 사용해보는 것은 어떨까요?\n"
    else:
        feedback += "- 답변에 더 긍정적인 표현을 사용해 보는 것이 좋겠습니다.\n"

    # 3. 구체성 관련 피드백
    feedback += "\n3. 구체성 분석:\n"
    if len(analysis_result['meaningful_words']) > 30:
        feedback += "- 답변이 상당히 구체적이고 상세합니다. 좋은 인상을 줄 수 있습니다.\n"
    elif len(analysis_result['meaningful_words']) > 20:
        feedback += "- 적절한 수준의 구체성을 보여주고 있습니다.\n"
    else:
        feedback += "- 답변을 조금 더 구체화하면 좋겠습니다. 특정 경험이나 예시를 추가해보세요.\n"

    # 4. 언어 사용 관련 피드백
    feedback += "\n4. 언어 사용 분석:\n"
    unique_words = set(analysis_result['meaningful_words'])
    if len(unique_words) > 30:
        feedback += "- 다양한 어휘를 사용하여 풍부한 답변을 제시했습니다. 이는 해당 분야에 대한 깊은 이해를 보여줍니다.\n"
    elif len(unique_words) > 20:
        feedback += "- 적절한 수준의 어휘를 사용하고 있습니다.\n"
    else:
        feedback += "- 더 다양한 전문 용어와 표현을 사용하여 답변을 풍부하게 만들어보세요.\n"

    return feedback

In [19]:
# 종합 피드백 및 개선 제안을 생성하는 함수 (sttresult1,sttresult2으로 던져줄 값)
def generate_comprehensive_feedback(analysis_result):
    job = "전체" if analysis_result['job'] == "전 직무" else analysis_result['job']
    feedback = f"이 답변은 전반적으로 {job} 직무와 관련된 경험과 기술을 잘 보여주고 있습니다.\n"
    feedback += f"앞으로의 면접에서는 {job} 직무에 중요한 기술이나 역량을 어떻게 개발하고 적용했는지 구체적인 예시와 함께 설명하고, 긍정적이고 열정적인 태도를 유지하면서 전문성을 잘 드러내는 어휘를 사용하세요."

    return feedback

In [20]:
# AI 모의 면접을 시작하는 함수 (수정됨)
def interview_simulation():
    print("AI 모의 면접을 시작합니다. '종료'라고 입력하면 면접이 종료됩니다.")
    while True:
        user_input = input("\n답변을 입력하세요: ")
        if user_input.lower() == '종료':
            print("면접이 종료되었습니다. 수고하셨습니다.")
            break

        analysis = analyze_response(user_input)
        feedback = generate_feedback(analysis)
        comprehensive_feedback = generate_comprehensive_feedback(analysis)

        print("\n피드백:")
        print(feedback)
        print("\n종합 평가 및 개선 제안:")
        print(comprehensive_feedback)

# 실행
if __name__ == "__main__":
    interview_simulation()

AI 모의 면접을 시작합니다. '종료'라고 입력하면 면접이 종료됩니다.

답변을 입력하세요: 최근 직장에서 겪었던 가장 큰 어려움은 팀 프로젝트의 마감 기한이 촉박할 때 발생했습니다. 프로젝트 초기 단계에서 예상보다 많은 문제가 발생하여 일정이 지연되었고, 팀원들 사이에 스트레스가 쌓였습니다. 이를 해결하기 위해 먼저 팀원들과의 회의를 통해 문제를 명확히 파악하고, 각자의 역할과 책임을 재조정했습니다. 또한, 우선순위를 정해 중요한 작업에 집중하도록 했습니다. 그 결과, 팀원들이 더 효율적으로 협력할 수 있었고, 최종적으로 마감 기한 내에 프로젝트를 성공적으로 완료할 수 있었습니다. 이 경험을 통해 팀워크와 커뮤니케이션의 중요성을 다시 한번 깨닫게 되었습니다.

피드백:
답변 카테고리: 태도 관련 역량
관련 역량/특성: 책임감 (표현: f_resp)
관련 직무: 제조/생산

1. 내용 분석:
- 다음 키워드들도 언급되어 다양한 역량을 보여주었습니다: 경험, 스트레스, 협력
- 제조/생산 직무와 더 관련성을 높이기 위해 다음 키워드들을 고려해보세요: 교대근무, 책임감, 안전

2. 태도 분석:
- 긍정적인 태도가 느껴집니다.

3. 구체성 분석:
- 답변이 상당히 구체적이고 상세합니다. 좋은 인상을 줄 수 있습니다.

4. 언어 사용 분석:
- 다양한 어휘를 사용하여 풍부한 답변을 제시했습니다. 이는 해당 분야에 대한 깊은 이해를 보여줍니다.


종합 평가 및 개선 제안:
이 답변은 전반적으로 제조/생산 직무와 관련된 경험과 기술을 잘 보여주고 있습니다.
앞으로의 면접에서는 제조/생산 직무에 중요한 기술이나 역량을 어떻게 개발하고 적용했는지 구체적인 예시와 함께 설명하고, 긍정적이고 열정적인 태도를 유지하면서 전문성을 잘 드러내는 어휘를 사용하세요.

답변을 입력하세요: 제가 겪었던 실패는 신규 고객 관리 시스템 도입 프로젝트에서 발생했습니다. 사용자 요구사항을 충분히 반영하지 않아 시스템이 실제 업무에 맞지 않았고, 이로 인해 수정 작업이 필요해 프로젝트가 지